In [10]:
# Imports
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [48]:

# URL
url = "https://www.numbeo.com/cost-of-living/rankings_by_country.jsp"

#get the webpage
page = requests.get(url)
dataset = {}

#get the dates available from the page selector
selectorName = "select[name='title']"
bsObj = bs(page.content, "html.parser")
#get the item with name title
selector = bsObj.select(selectorName)
#extract the dates
dateRegex = "option value=\"(.*?)\""
dates = re.findall(dateRegex, str(selector))

for date in dates:

    if "mid" not in date:

            newPage = requests.get(url + "?title=" + date)
            bsObj = bs(newPage.content, "html.parser")
            dataset[date] = {}

            # select the table body
            tableSelector = "tbody"
            tableBody = bsObj.select(tableSelector)

            # select the table rows
            table = tableBody[0].select("tr")

            # split the table by <tr style="width: 100%">
            # each row has the following columns: cityOrCountryInIndicesTable=Country, Cost of Living, Rent Index, Cost of Living Plus Rent Index, Groceries Index, Restaurant Price Index, Local Purchasing Power Index
            # extract the data from the table
            for i in table:
                splitData = str(i).split("\n")[2:-1]
                dataRegex = "<td .*?>(.*?)<\/td>"
                data = re.findall(dataRegex, str(i))
                countryName = data[0]
                costOfLiving = data[1]
                rentIndex = data[2]
                costOfLivingPlusRentIndex = data[3]
                groceriesIndex = data[4]
                restaurantPriceIndex = data[5]
                localPurchasingPowerIndex = data[6]
                dataset[date][countryName] = [costOfLiving, rentIndex, costOfLivingPlusRentIndex, groceriesIndex, restaurantPriceIndex, localPurchasingPowerIndex]

#create the csv file to store the data
file = open("costOfLiving.csv", "w")
file.write("Date,Country,Cost of Living,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index\n")
for date in dataset:
    for country in dataset[date]:
        file.write(date + "," + country + "," + ",".join(dataset[date][country]) + "\n")
file.close()
print("Done")


Done
